<a href="https://colab.research.google.com/github/BooEiEi/project/blob/main/multiple_imputation3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd #นำเข้า pandas แทนที่ด้วย pd
import os# os ใช้จัดการกับไฟล์ ว่าอยู่ที่ไหน
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive #นำเข้าข้อมูลจาก drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/project'# ชี้ dirve ที่เราต้องการ
import missingno as mno
import missingno as msno
from sklearn import linear_model
%matplotlib inline
import matplotlib
import sklearn 
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
import time

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv(os.path.join(path, 'DFforCreateNaN.csv'))
data = data.drop(['Unnamed: 0'], axis=1)
data

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


# random NaN 

In [ ]:
X = data.drop(['Sex', 'AgeSYear', '002_BUN', '003_Creatinine', '005_Cholesterol', '006_Triglyceride', '008_HDL-C', '009_LDL', '010_AST'], axis=1)

In [ ]:
y= data.loc[:,['Sex', 'AgeSYear', '002_BUN', '003_Creatinine', '005_Cholesterol', '006_Triglyceride', '008_HDL-C', '009_LDL', '010_AST']]

In [ ]:
data1=y.join(X)
data1

,Sex,AgeSYear,002_BUN,003_Creatinine,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,Weight,...,Fat Mass,% Body fat,waist to hip radio,abdominal circumference,visceral fat area,001_Glucose,004_Uric Acid,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,15.0,1.0,176.0,51.0,72.0,94.0,16.0,47.4,...,12.5,26.4,0.80,70.6,43.0,81.0,5.0,16.0,62.0,5.1
1,2,24.0,12.0,0.8,159.0,45.0,52.0,98.0,13.0,53.2,...,14.6,27.4,0.77,73.8,39.0,79.0,5.2,17.0,50.0,5.4
2,2,62.0,17.0,0.7,230.0,94.0,61.0,151.0,18.0,54.4,...,15.2,27.9,0.83,73.8,7.0,89.0,4.1,31.0,76.0,5.6
3,2,35.0,10.0,0.6,198.0,47.0,68.0,120.0,16.0,56.3,...,15.4,27.4,0.78,74.0,42.0,84.0,5.0,20.0,35.0,5.1
4,1,46.0,14.0,0.9,194.0,63.0,60.0,122.0,15.0,72.3,...,19.0,26.4,0.93,91.4,136.0,78.0,6.9,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,14.0,0.9,182.0,179.0,39.0,107.0,36.0,72.0,...,13.8,20.1,0.93,84.0,109.0,92.0,6.9,31.0,79.0,6.5
149,1,58.0,18.0,1.0,191.0,62.0,47.0,132.0,16.0,62.0,...,11.9,19.2,0.92,81.3,101.0,102.0,6.8,33.0,42.0,6.8
150,2,42.0,9.0,0.8,212.0,68.0,63.0,135.0,11.0,55.5,...,14.1,25.4,0.77,72.5,37.0,81.0,4.7,16.0,33.0,4.8
151,2,61.0,12.0,0.8,235.0,66.0,56.0,166.0,15.0,61.8,...,19.2,31.1,0.84,78.5,72.0,83.0,5.1,23.0,44.0,5.4


In [ ]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'X{i}'] = X.copy()
  np.random.seed(i)
  mask = np.zeros(153*17, dtype=bool)
  n = int((153*17)*(16/100)) 
  mask[:n] = True
  np.random.shuffle(mask)
  mask = mask.reshape(153, 17)
  globals()[f'X{i}'][mask] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 1.460543155670166 วินาที
เวลาที่ใช้ในการประมวลผล 0.0243423859278361 นาที


In [ ]:
X101

,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,% Body fat,waist to hip radio,abdominal circumference,visceral fat area,001_Glucose,004_Uric Acid,012_ALT,013_Alkaline Phos,071_HbA1c
0,47.4,153.0,20.2,NaN,75.0,69.0,NaN,NaN,26.4,0.80,NaN,43.0,81.0,5.0,16.0,62.0,NaN
1,53.2,157.0,21.6,103.0,69.0,NaN,21.3,14.6,27.4,0.77,73.8,39.0,79.0,5.2,17.0,50.0,NaN
2,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,27.9,0.83,NaN,7.0,89.0,NaN,NaN,76.0,5.6
3,56.3,161.0,21.7,95.0,57.0,72.0,22.5,NaN,27.4,0.78,74.0,42.0,84.0,5.0,20.0,35.0,5.1
4,72.3,NaN,NaN,133.0,NaN,107.0,NaN,19.0,26.4,0.93,91.4,136.0,78.0,6.9,NaN,NaN,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,72.0,170.0,24.9,131.0,74.0,69.0,30.5,NaN,20.1,0.93,84.0,NaN,92.0,6.9,NaN,79.0,6.5
149,62.0,168.0,22.0,103.0,NaN,72.0,27.8,11.9,19.2,0.92,81.3,101.0,102.0,6.8,33.0,42.0,6.8
150,55.5,165.0,20.4,102.0,52.0,77.0,NaN,14.1,25.4,0.77,72.5,37.0,81.0,4.7,16.0,33.0,4.8
151,61.8,159.0,NaN,117.0,60.0,72.0,23.4,19.2,31.1,0.84,78.5,NaN,83.0,5.1,23.0,44.0,NaN


In [ ]:
for i in range(1,1001):
  globals()[f'data_random{i}'] = y.join(globals()[f'X{i}'])

In [ ]:
data_random10

,Sex,AgeSYear,002_BUN,003_Creatinine,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,Weight,...,Fat Mass,% Body fat,waist to hip radio,abdominal circumference,visceral fat area,001_Glucose,004_Uric Acid,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,15.0,1.0,176.0,51.0,72.0,94.0,16.0,47.4,...,12.5,26.4,NaN,70.6,43.0,NaN,5.0,NaN,62.0,5.1
1,2,24.0,12.0,0.8,159.0,45.0,52.0,98.0,13.0,53.2,...,NaN,NaN,NaN,73.8,39.0,79.0,5.2,17.0,NaN,5.4
2,2,62.0,17.0,0.7,230.0,94.0,61.0,151.0,18.0,54.4,...,15.2,27.9,0.83,73.8,7.0,89.0,4.1,NaN,76.0,5.6
3,2,35.0,10.0,0.6,198.0,47.0,68.0,120.0,16.0,56.3,...,NaN,27.4,0.78,NaN,42.0,84.0,5.0,20.0,35.0,5.1
4,1,46.0,14.0,0.9,194.0,63.0,60.0,122.0,15.0,72.3,...,NaN,26.4,0.93,91.4,136.0,78.0,6.9,19.0,69.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,14.0,0.9,182.0,179.0,39.0,107.0,36.0,NaN,...,13.8,20.1,0.93,84.0,109.0,92.0,6.9,31.0,79.0,NaN
149,1,58.0,18.0,1.0,191.0,62.0,47.0,132.0,16.0,62.0,...,11.9,19.2,0.92,NaN,101.0,102.0,6.8,33.0,42.0,6.8
150,2,42.0,9.0,0.8,212.0,68.0,63.0,135.0,11.0,NaN,...,14.1,25.4,0.77,72.5,37.0,81.0,NaN,16.0,33.0,NaN
151,2,61.0,12.0,0.8,235.0,66.0,56.0,166.0,15.0,NaN,...,NaN,NaN,0.84,78.5,72.0,83.0,5.1,23.0,NaN,NaN


# Imputation

## Miss Forest

In [ ]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error
     

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF10_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf_{i}'] = MF10_imputer.fit_transform(globals()[f'data_random{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF_{i}'] = mean_squared_error(data1, globals()[f'imputed_mf_{i}']) # for loop mean_squared_error
t_end = time.time()
t_MF= (t_end- t_start)
     

In [25]:
t_MF

25080.002326965332

In [26]:
t_start = time.time()
mse_mf_total = 0 
for i in range(1,1001):
  mse_mf_total = mse_mf_total + globals()[f'MSE_MF_{i}']
AMSE_mf = mse_mf_total/1000
print(f'AMSE_mf = {AMSE_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')
     

AMSE_mf = 4.711254129338661
เวลาที่ใช้ในการประมวลผล 0.00618743896484375 วินาที
เวลาที่ใช้ในการประมวลผล 0.00010312398274739584 นาที


## Imputation using the KNN

In [27]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [28]:
t_start = time.time()
KNN_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN_{i}'] = KNN_imputer.fit_transform(globals()[f'data_random{i}'].copy())
  globals()[f'MSE_KNN_{i}'] = mean_squared_error(data1, globals()[f'imputed_KNN_{i}'])
t_end = time.time()
t_KNN = (t_end- t_start)

In [29]:
t_KNN

36.725574016571045

In [30]:
t_start = time.time()
mse_knn_total = 0 
MSE_KNN1001 = 0
for i in range(1,1001):
  mse_knn_total = mse_knn_total + globals()[f'MSE_KNN_{i}']
AMSE_knn = mse_knn_total/1000
print(f'AMSE_knn = {AMSE_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_knn = 10.044105013132006
เวลาที่ใช้ในการประมวลผล 0.005882978439331055 วินาที
เวลาที่ใช้ในการประมวลผล 9.804964065551758e-05 นาที


## Imputation using the MICE

In [31]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr_{i}'] = imp.fit_transform(globals()[f'data_random{i}'].copy())
  globals()[f'MSE_lr_{i}'] = mean_squared_error(data1, globals()[f'imputed_lr_{i}'])
t_end = time.time()
t_LR = (t_end- t_start)

In [33]:
t_LR

331.40486240386963

In [34]:
t_start = time.time()
mse_lr_total = 0 
MSE_lr_1001 = 0
for i in range(1,1001):
  mse_lr_total = mse_lr_total + globals()[f'MSE_lr_{i}']
AMSE_lr = mse_lr_total/1000
print(f'AMSE_lr = {AMSE_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_lr = 11.854957936318835
เวลาที่ใช้ในการประมวลผล 0.0010976791381835938 วินาที
เวลาที่ใช้ในการประมวลผล 1.8294652303059896e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm_{i}'] = svm_svr.fit_transform(globals()[f'data_random{i}'].copy())
  globals()[f'MSE_svm_{i}'] = mean_squared_error(data1, globals()[f'imputed_svm_{i}'])
t_end = time.time()
t_SVM = (t_end- t_start)

In [36]:
t_SVM

171.3664848804474

In [37]:
t_start = time.time()
mse_svm_total = 0 
MSE_svm_1001 = 0
for i in range(1,1001):
  mse_svm_total = mse_svm_total + globals()[f'MSE_svm_{i}']
AMSE_svm = mse_svm_total/1000
print(f'AMSE_svm = {AMSE_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_svm = 13.254521159461955
เวลาที่ใช้ในการประมวลผล 0.0030798912048339844 วินาที
เวลาที่ใช้ในการประมวลผล 5.133152008056641e-05 นาที


## Imputation using the EM

### impy.em

In [38]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy_{i}'] = impy.em(globals()[f'data_random{i}'].values.copy())
  globals()[f'MSE_impy_{i}'] = mean_squared_error(data1, globals()[f'imputed_impy_{i}'])
t_end = time.time()
t_EM = (t_end- t_start)

In [40]:
t_EM

163.02992725372314

In [41]:
t_start = time.time()
mse_impy_total = 0 
MSE_impy_1001 = 0
for i in range(1,1001):
  mse_impy_total = mse_impy_total + globals()[f'MSE_impy_{i}']
AMSE_impy = mse_impy_total/1000
print(f'AMSE_impy = {AMSE_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_impy = 24.78057979739025
เวลาที่ใช้ในการประมวลผล 0.0012564659118652344 วินาที
เวลาที่ใช้ในการประมวลผล 2.094109853108724e-05 นาที
